In [ ]:
from scapy.all import *
import numpy as np

In [ ]:
def ReadCapture(capture_name, timestamps_filename):
    Relevant_Packets = []
    with PcapReader(capture_name) as pcap_reader:
        for pkt in pcap_reader:
            if IP in pkt:
                ip_src = pkt[IP].src
                if (ip_src == "10.10.0.2" and pkt.haslayer(UDP) and pkt[UDP].len > 140 and pkt[UDP].len < 1350
                    and (not pkt.haslayer(ICMP))):
                    Relevant_Packets.append(pkt)
                    with open(timestamps_filename, "a") as file:
                        file.write(str(pkt.time) + "\n")
    return Relevant_Packets

In [ ]:
def Packet_Timestamp_Mapping_Server(timestamps_filename, packets_filename):
    timestamps = np.loadtxt(timestamps_filename)
    timestamps = timestamps.tolist()
    with open(packets_filename, "r") as file:
        packets = file.readlines()
    Packet_Timestamp = {}
    for i in range(len(packets)):
        pkt_id = int(packets[i].split(":")[0])
        Packet_Timestamp[pkt_id] = timestamps[i]
    return Packet_Timestamp

In [ ]:
def Packet_Timestamp_Mapping_Client(timestamps_filename):
    timestamps = np.loadtxt(timestamps_filename)
    timestamps = timestamps.tolist()
    Packet_Timestamp = {}
    
    Packet_Timestamp[1] = timestamps[0]
    last_valid_timestamp = timestamps[0]
    last_entry = 1
    i = 1

    while(i < len(timestamps)):
        expected_timestamp = last_valid_timestamp + 0.1
        min_diff = 100000
        for j in range(i, len(timestamps)):
            new_diff = np.abs(expected_timestamp - timestamps[j])
            if (new_diff < min_diff and j < len(timestamps) - 1):
                min_diff = new_diff

            elif new_diff >= min_diff:
                i = j
                last_entry += 1
                last_valid_timestamp = timestamps[j-1]
                Packet_Timestamp[last_entry] = last_valid_timestamp
                break
            else:
                last_entry +=1
                Packet_Timestamp[last_entry] = timestamps[j]
                print("Last entry: " + str(last_entry) + ", " + str(Packet_Timestamp[last_entry]))
                break
        if (last_entry == 100000):
            break
    return Packet_Timestamp

In [ ]:
def main(loss, capture_path):
    capture_name_client =  capture_path + "captured-packets_client.pcap"
    capture_name_server = capture_path + "captured-packets_server.pcap"
    Relevant_Packets_client = ReadCapture(capture_name_client, "NetworkDelay_client_" + str(loss) + ".txt")
    Relevant_Packets_server = ReadCapture(capture_name_server, "NetworkDelay_server_" + str(loss) + ".txt")

In [ ]:
def GetNetworkDelay(Packet_Timestamp_client, Packet_Timestamp_server):
    Delays = []
    for pkt_id in Packet_Timestamp_server.keys():
        delay = Packet_Timestamp_server[pkt_id] - Packet_Timestamp_client[pkt_id]
        if (delay < 0):
            print(delay, pkt_id)
            break
        Delays.append(delay)
    Delays = np.array(Delays)
    print("MEAN: " + str(np.mean(Delays)*1000))
    print("MAX: " + str(np.max(Delays)*1000))
    print("MIN: " + str(np.min(Delays)*1000))
    print("MEDIAN: " + str(np.median(Delays)*1000))

In [ ]:
loss=5
Network_Delay_path_server = "NetworkDelay/Modified/Improved/Loss=" + str(loss) +"/NetworkDelay_server.txt"
Packet_Timestamp_server = Packet_Timestamp_Mapping_Server(Network_Delay_path_server, "Improved/Modified/Loss=5/delay_client_server.txt")
Network_Delay_path_client = "NetworkDelay/Modified/Improved/Loss=" + str(loss) +"/NetworkDelay_client.txt"
Packet_Timestamp_client = Packet_Timestamp_Mapping_Client(Network_Delay_path_client)
GetNetworkDelay(Packet_Timestamp_client, Packet_Timestamp_server)

In [ ]:
loss=20
Network_Delay_path_server = "NetworkDelay/Default/Improved/Loss=" + str(loss) +"_2/NetworkDelay_server.txt"
Packet_Timestamp_server = Packet_Timestamp_Mapping_Server(Network_Delay_path_server, "Improved/Default/Loss=20_2/delay_client_server.txt")
Network_Delay_path_client = "NetworkDelay/Default/Improved/Loss=" + str(loss) +"_2/NetworkDelay_client.txt"
Packet_Timestamp_client = Packet_Timestamp_Mapping_Client(Network_Delay_path_client)
GetNetworkDelay(Packet_Timestamp_client, Packet_Timestamp_server)

In [ ]:
loss=4
Network_Delay_path_server = "NetworkDelay/Modified/Improved/Loss=" + str(loss) +"/NetworkDelay_server.txt"
Packet_Timestamp_server = Packet_Timestamp_Mapping_Server(Network_Delay_path_server, "Improved/Modified/Loss=4/delay_client_server.txt")
Network_Delay_path_client = "NetworkDelay/Modified/Improved/Loss=" + str(loss) +"/NetworkDelay_client.txt"
Packet_Timestamp_client = Packet_Timestamp_Mapping_Client(Network_Delay_path_client)
GetNetworkDelay(Packet_Timestamp_client, Packet_Timestamp_server)